In [1]:
!pip install -q openai

You should consider upgrading via the '/Users/michaelryaboy/recent-projects/inference-webhook/venv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import os, json, io, time, random
from openai import OpenAI

N = 10_000
client = OpenAI(
    base_url="https://api.inference.net/v1",
    api_key=os.getenv("INFERENCE_API_KEY"),
)

buf = io.BytesIO()
for i in range(N):
    buf.write(json.dumps({
        "custom_id": f"req-{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "meta-llama/llama-3.2-1b-instruct/fp-8",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content":
                 f"What is {random.choice(['2+2', 'the capital of France', 'the opposite of hot'])}?"},
            ],
            "max_tokens": 5,
        },
    }).encode() + b'\n')
buf.seek(0)
buf.name = "batch.jsonl"          # <<‑ add a filename so the API sees “.jsonl”

file_id = client.files.create(file=buf, purpose="batch").id
batch   = client.batches.create(
    input_file_id=file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)
print("Batch ID:", batch.id)

while batch.status not in {"completed", "failed", "expired", "cancelled"}:
    time.sleep(10)
    batch = client.batches.retrieve(batch.id)
    print("status:", batch.status, batch.request_counts)

if batch.status == "completed":
    out = client.files.content(batch.output_file_id).text.splitlines()
    print("\nSample output:\n", "\n".join(out[:5]))
else:
    print("Batch ended with status:", batch.status)


Batch ID: hBrvLzemNvx8hzb0mXQ1O
status: in_progress BatchRequestCounts(completed=0, failed=0, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=18, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=766, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=1287, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=1591, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=2072, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=2619, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=3023, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=3349, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=3981, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=4396, total=10000)
status: in_progress BatchRequestCounts(completed=0, failed=4762, total=10000)
status: in_progress BatchRequestCounts